El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

In [47]:
#Instalar librerías
#!pip install lightgbm #Al completar la instalación se puede comentar. 

#Importación de librerías
import pandas as pd
import numpy as np
import time

#Modelos, métricas y manipulación de datos para modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
import lightgbm as lgb

## Preparación de datos

In [48]:
car_df = pd.read_csv("car_data.csv")
car_df.info()
display(car_df.head())
print("Las variables de fechas son objetos {DateCrawled, DateCreated, LastSeen} preferible cambiarlas a su formato: datetime. ")

#Cambio de tipo a datetime. 
columnas_fecha = ["DateCrawled", "DateCreated", "LastSeen"]
for col in columnas_fecha: 
    car_df[col] = pd.to_datetime(car_df[col], format = "%d/%m/%Y %H:%M") #No hay nulos por lo tanto no se ocupa el argumento errors= "coerce"
    
#Revisión de nuevos tipos de datos con .info() 
car_df.info()
print(f"Tipos de datos de {columnas_fecha} cambiados a formato datetime")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


Las variables de fechas son objetos {DateCrawled, DateCreated, LastSeen} preferible cambiarlas a su formato: datetime. 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Mileage            354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  NotRepaired        283215 non-null  

In [49]:
#Revisión de nulos 
print(car_df.isna().mean())
print("Hay nulos en 5 variables {VehicleType, Gearbox, Model, Fueltype, NotRepaired}. Todas las variables son objetos o categóricas, los valores pueden ser cambiados por no registrado en lugar de valores nulos para facilitar el entrenado de los modelos.")

#Cuál es el rango de los precios. Importante conocer sobre la variable objetivo. 
precio_minimo = car_df["Price"].min()
precio_maximo = car_df["Price"].max()
rango_precios = car_df["Price"].max() - car_df["Price"].min()
promedio_precios = car_df["Price"].mean()
mediana_precios = car_df["Price"].median()
ceros_precio = (car_df["Price"]== 0).sum()

print(f"Precio máximo: {precio_maximo} dólares \nPrecio mínimo: {precio_minimo} dólares \nLLegan a variar en {rango_precios} dólares")
print("\nHay carros de 0 dólares. Interesante, debe haber habido nulos manejados como 0")
print(f"Prmedio de precios de carros: {promedio_precios} dólares \nMediana de precios de carros: {mediana_precios} dólares")
print(f"Un total de {ceros_precio} carros con registro de precio de 0 dólares.") #10772 carros de precio 0. 

DateCrawled          0.000000
Price                0.000000
VehicleType          0.105794
RegistrationYear     0.000000
Gearbox              0.055967
Power                0.000000
Model                0.055606
Mileage              0.000000
RegistrationMonth    0.000000
FuelType             0.092827
Brand                0.000000
NotRepaired          0.200791
DateCreated          0.000000
NumberOfPictures     0.000000
PostalCode           0.000000
LastSeen             0.000000
dtype: float64
Hay nulos en 5 variables {VehicleType, Gearbox, Model, Fueltype, NotRepaired}. Todas las variables son objetos o categóricas, los valores pueden ser cambiados por no registrado en lugar de valores nulos para facilitar el entrenado de los modelos.
Precio máximo: 20000 dólares 
Precio mínimo: 0 dólares 
LLegan a variar en 20000 dólares

Hay carros de 0 dólares. Interesante, debe haber habido nulos manejados como 0
Prmedio de precios de carros: 4416.656775846645 dólares 
Mediana de precios de carros: 27

## Entrenamiento del modelo 

In [ ]:
#Generar funciones para métricas de calidad aplicable al interior de funciones de modelos. 
#Error cuadrático medio 
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [ ]:
#Estandarización e imputación de los datos. 

#Me di cuenta que en los precios de algunos carros hay registro de valor 0 no creo sea útil predecir ausencia de valor en los carros supongo no hubo registro eliminaré esos datos. 
car_df.columns #Price es el objetivo.
car_df_sin_ceros = car_df[car_df["Price"] != 0] 
car_df_objetivo = car_df_sin_ceros["Price"]
car_df_caracteristicas = car_df_sin_ceros.drop(columns="Price")
car_df_caracteristicas = car_df_caracteristicas.drop(columns=columnas_fecha) #Remoción de datos tipo fecha. Datetime no son admisibles para modelos de aprendizaje

#Imputación de valores faltantes en variables categóricas
car_df_caracteristicas = car_df_caracteristicas.fillna("unregistered")

#Para este momento los datos ya están separados por objetivo y características. 

#Los datos son 354369 filas. Para reducir los tiempos de búsqueda de hiperparámetros se puede hacer un muestreo. 
x_sample, _, y_sample, _ = train_test_split(car_df_caracteristicas, car_df_objetivo, train_size=0.2, random_state=54321) #x_sample es la muestra de 20% de las características e y_sample del objetivo.  

#Separación de los datos en 60% entrenamiento, 20% test, 20% validación. 
df_cars_entrenamiento, df_cars_test_valid, df_objetivo_entrenamiento, df_objetivo_test_valid = train_test_split(car_df_caracteristicas, car_df_objetivo, train_size=0.6, random_state=54321) #Se dividen los datos en entrenamiento y conjunto para dividir en prueba y validación. Cuenta como 60% entrenamiento, 40% prueba también.

df_cars_test, df_cars_valid, df_objetivo_test, df_objetivo_valid = train_test_split(df_cars_test_valid, df_objetivo_test_valid, train_size=0.5, random_state=54321) #Salen 2 sets: validación y prueba de 20% del tamaño total. 50% de 40%. Solo por si quiero validar. 

#Hacer una segregación por tipo de datos. 
columnas_numericas = car_df_caracteristicas.select_dtypes(include=["int64", "float64"]).columns.to_list()
columnas_categoricas = car_df_caracteristicas.select_dtypes(include="object").columns.to_list()

#Preprocesadores separados 
preprocesamiento_numerico = Pipeline([("escalar", StandardScaler())])
preprocesamiento_ohe = OneHotEncoder(handle_unknown="ignore")
preprocesamiento_ordinal = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [52]:
#Preprocesador a aplicar para búsqueda de hiperparámetros para la regresión lineal 
preprocesador_lr = ColumnTransformer([
    ("num", preprocesamiento_numerico, columnas_numericas),
    ("cat", preprocesamiento_ohe, columnas_categoricas)
])

pipeline_lr = Pipeline([
    ("preprocesador", preprocesador_lr),
    ("modelo", LinearRegression())
])

param_grid_lr = {
    "modelo__fit_intercept": [True, False]
}

grid_lr = GridSearchCV(
    pipeline_lr,
    param_grid_lr,
    scoring=rmse_scorer,
    cv=3
)

grid_lr.fit(x_sample, y_sample)
print("Linear Regression - Best RMSE (CV):", -grid_lr.best_score_)
print("Best Params:", grid_lr.best_params_)
print(f"Los resultados de la regresión lineal previo a entrenar y poner a prueba, es decir por validación cruzada, llega a variar en {-grid_lr.best_score_:.2f} dolares.")

#grid_lr.best_estimator_ sostiene los mejores valores de hiperparámetros y el modelo para posteriormente entrenar el mejor modelo con los datos completos en lugar de validación cruzada. Aplica de forma similar para todos los modelos en el que se use GridSearchCV. 

Linear Regression - Best RMSE (CV): 3130.1289877705008
Best Params: {'modelo__fit_intercept': True}
Los resultados de la regresión lineal previo a entrenar y poner a prueba, es decir por validación cruzada, llega a variar en 3130.13 dolares.


In [ ]:
#Prueba de cordura: Regresión Lineal. Poner a prueba la regresión con los datos de entrenamiento y prueba. 
mejor_modelo_lr = grid_lr.best_estimator_
start_train_lr = time.time()
mejor_modelo_lr.fit(df_cars_entrenamiento, df_objetivo_entrenamiento)
tiempo_entrenamiento_lr = time.time() -start_train_lr

start_pred_lr = time.time()
obj_prediccion = mejor_modelo_lr.predict(df_cars_test_valid)
tiempo_pred_lr = time.time()-start_pred_lr
rmse_lr = np.sqrt(mean_squared_error(df_objetivo_test_valid, obj_prediccion))
rs = r2_score(df_objetivo_test_valid, obj_prediccion)

print(f"RECM en prueba:{rmse_lr:.2f} dólares. El RECM es similar a aquel obtenido en datos de muestreo.")
print(f"r2 en prueba: {rs}")
print("El modelo parece tener una tendencia practicamente aleatoria respecto a la predicción de los datos. ¿Qué tanta mejora habrá en otros modelos?")
print(f"Tiempo de entrenamiento de regresión lineal: {tiempo_entrenamiento_lr:.2f} segundos,\nTiempo de predicción de regresión lineal: {tiempo_pred_lr:.2f} segundos")

RECM en prueba:3130.65 dólares. El RECM es similar a aquel obtenido en datos de muestreo.
r2 en prueba: 0.5208859904512142
El modelo parece tener una tendencia practicamente aleatoria respecto a la predicción de los datos. ¿Qué tanta mejora habrá en otros modelos?
Tiempo de entrenamiento de regresión lineal: 3.52 segundos,
Tiempo de predicción de regresión lineal: 0.24 segundos


In [ ]:
#Preprocesador para la búsqueda de hiperparámetros para el bosque aleatorio
preprocesador_rf = ColumnTransformer([
    ("num", preprocesamiento_numerico, columnas_numericas),
    ("cat", preprocesamiento_ordinal, columnas_categoricas)
])

pipeline_rf = Pipeline([
    ("preprocesador", preprocesador_rf),
    ("modelo", RandomForestRegressor(random_state=42))
])

param_grid_rf = {
    "modelo__n_estimators": [100, 200],
    "modelo__max_depth": [5, 10],
    "modelo__min_samples_split": [2, 5]
}

grid_rf = GridSearchCV(
    pipeline_rf,
    param_grid_rf,
    scoring= rmse_scorer,
    cv=3,
    n_jobs=-1
)

grid_rf.fit(x_sample, y_sample)
print("Mejor RECM arbol aleatorio por (CV):", -grid_rf.best_score_) #con datos de muestreo
print("Best Params:", grid_rf.best_params_)
print("El bosque aleatorio llegó a un RECM menor por 1100 dólares en datos de muestreo. Mejor desempeño que la regresión lineal")

Mejor RECM arbol aleatorio por (CV): 1986.8380262554117
Best Params: {'modelo__max_depth': 10, 'modelo__min_samples_split': 2, 'modelo__n_estimators': 200}
El bosque aleatorio llegó a un RECM menor por 1100 dólares en datos de muestreo. Mejor desempeño que la regresión lineal


In [55]:
#Bosque aleatorio
mejor_modelo_bosque_aleatorio = grid_rf.best_estimator_
start_train_rf = time.time()
mejor_modelo_bosque_aleatorio.fit(df_cars_entrenamiento, df_objetivo_entrenamiento)
tiempo_entrenamiento_rf = time.time() -start_train_rf

start_pred_rf = time.time()
objetivo_prediccion_rf = mejor_modelo_bosque_aleatorio.predict(df_cars_test_valid)
tiempo_pred_rf = time.time() - start_pred_rf
rmse_rf = np.sqrt(mean_squared_error(df_objetivo_test_valid, objetivo_prediccion_rf))
print(f"RECM de la prueba con el mejor modelo de bosque aleatorio en los valores de prueba: {rmse_rf:.2f}")
print(f"Tiempo de entrenamiento de bosquealeatorio: {tiempo_entrenamiento_rf:.2f} segundos,\nTiempo de predicción de bosque aleatorio: {tiempo_pred_rf:.2f} segundos")

RECM de la prueba con el mejor modelo de bosque aleatorio en los valores de prueba: 1945.76
Tiempo de entrenamiento de bosquealeatorio: 93.02 segundos,
Tiempo de predicción de bosque aleatorio: 2.34 segundos


In [56]:
#Preprocesador para la búsqueda de hiperparámetros para la potenciación de gradiente. Lightgbm
pipeline_lgbm = Pipeline([
    ("preprocesador", preprocesador_rf),  # usamos el mismo Ordinal
    ("modelo", lgb.LGBMRegressor(random_state=42))
])

param_grid_lgbm = {
    "modelo__n_estimators": [100, 200],
    "modelo__learning_rate": [0.05, 0.1],
    "modelo__num_leaves": [31, 50]
}

grid_lgbm = GridSearchCV(
    pipeline_lgbm,
    param_grid_lgbm,
    scoring=rmse_scorer,
    cv=3,
    n_jobs=-1
)

grid_lgbm.fit(x_sample, y_sample)
print("LightGBM - Best RMSE (CV):", -grid_lgbm.best_score_)
print("Best Params:", grid_lgbm.best_params_)
print("\nSe observó una mejora de 150 dólares de reducción de error de tipo RECM usando Lightgbm comparado con bosque aleatorio en datos de muestreo.")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 923
[LightGBM] [Info] Number of data points in the train set: 68719, number of used features: 11
[LightGBM] [Info] Start training from score 4553.577031
LightGBM - Best RMSE (CV): 1758.8636677551788
Best Params: {'modelo__learning_rate': 0.1, 'modelo__n_estimators': 200, 'modelo__num_leaves': 50}

Se observó una mejora de 150 dólares de reducción de error de tipo RECM usando Lightgbm comparado con bosque aleatorio en datos de muestreo.


In [57]:
#Potenciación del gradiente
mejor_modelo_lgbm = grid_lgbm.best_estimator_
start_train_lgbm = time.time()
mejor_modelo_lgbm.fit(df_cars_entrenamiento, df_objetivo_entrenamiento)
tiempo_entrenamiento_lgbm = time.time() - start_train_lgbm

start_pred_lgbm = time.time()
objetivo_prediccion_lgbm = mejor_modelo_lgbm.predict(df_cars_test_valid)
tiempo_pred_lgbm = time.time() - start_pred_lgbm
rmse_lgbm = np.sqrt(mean_squared_error(df_objetivo_test_valid, objetivo_prediccion_lgbm))
print(f"RECM de la prueba con el mejor modelo de potenciación del gradiente lgbm en los valores de prueba: {rmse_lgbm:.2f}")
print(f"Casi 300 dólares de reducción en error de predicción del valor a una velocidad mayor. Definitivamente el mejor modelo de predicción para la tarea.")
print(f"Tiempo de entrenamiento de lgbm: {tiempo_entrenamiento_lgbm},\nTiempo de predicción de lgbm: {tiempo_pred_lgbm}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 948
[LightGBM] [Info] Number of data points in the train set: 206158, number of used features: 11
[LightGBM] [Info] Start training from score 4548.766533


c:\Users\david\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


RECM de la prueba con el mejor modelo de potenciación del gradiente lgbm en los valores de prueba: 1672.29
Casi 300 dólares de reducción en error de predicción del valor a una velocidad mayor. Definitivamente el mejor modelo de predicción para la tarea.
Tiempo de entrenamiento de lgbm: 2.2061123847961426,
Tiempo de predicción de lgbm: 1.0693230628967285


## Análisis del modelo

In [ ]:
#Reporte comparativo de las métricas de calidad de predicción por modelo y el tiempo empleado por cada modelo en entrenamiento y predicción
tiempo_entrenamiento_lr, tiempo_pred_lr, tiempo_entrenamiento_rf, tiempo_pred_rf, tiempo_entrenamiento_lgbm, tiempo_pred_lgbm

nombres_modelos = ["Regresión Lineal", "Bosque Aleatorio", "Potenciación de gradiente lgbm"]
tiempos_entrenamiento = [tiempo_entrenamiento_lr, tiempo_entrenamiento_rf, tiempo_entrenamiento_lgbm]
tiempos_prediccion = [tiempo_pred_lr, tiempo_pred_rf, tiempo_pred_lgbm]
puntajes_recm = [rmse_lr, rmse_rf, rmse_lgbm]

datos_analisis_modelos = pd.DataFrame({
    "Modelos": nombres_modelos, 
    "Tiempo de Entrenamiento (s)": tiempos_entrenamiento, 
    "Tiempo en Predecir (s)": tiempos_prediccion, 
    "RECM": puntajes_recm
})

print("Reporte comparativo de las métricas de calidad de predicción por modelo y el tiempo empleado por cada modelo en entrenamiento y predicción:")
display(datos_analisis_modelos)

Reporte comparativo de las métricas de calidad de predicción por modelo y el tiempo empleado por cada modelo en entrenamiento y predicción:


,Modelos,Tiempo de Entrenamiento (s),Tiempo en Predecir (s),RECM
0,Regresión Lineal,3.516922,0.235065,3130.646704
1,Bosque Aleatorio,93.018013,2.341324,1945.763623
2,Potenciación de gradiente lgbm,2.206112,1.069323,1672.289291


Conclusiones

1. El modelo más efectivo en predecir el valor de un automovil basado en sus características fue: Pontenciación de gradiente Lightgbm. Una tendencia de 1672 dólares de error sobre la predicción.

2. El modelo que empleó la menor cantidad de tiempo en entrenamiento fue: Pontenciación de gradiente Lightgbm. 

3. El modelo que empleó la menor cantidad de tiempo en predecir fue: Regresión lineal. 0.2 segundos. Lightgbm tardó 1 segundo. 

4. Basado en las características de métricas de calidad y de velocidad el modelo más adecuado para desempeñar la tarea para Rusty Bargain es: Pontenciación de gradiente Lightgbm. Generando estimados de costo de los automóviles con un error estimado de 1672 dólares y tardando poco en ser entrenado y predecir. 